In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score, KFold

In [113]:
glogs_final = pd.read_csv('https://raw.githubusercontent.com/tmarchok1/DS440_project/refs/heads/main/glogs_final.csv')
glogs_final.head()

,date,year,week,day_of_week,v_name,h_name,day_night,park_id,temp,precip,capacity,prev_year_wins,made_playoffs,won_division,InstagramFollowers,CityPopulation,attendance
0,2005-04-03,2005,1,Sun,BOS,NYY,1,Old Yankee Stadium,51.7,0.587,56937.0,101,1,1,3900000,19940274,54818.0
1,2005-04-04,2005,1,Mon,OAK,BAL,0,Camden Yards,60.7,0.000,45971.0,78,0,0,746000,2859024,48271.0
2,2005-04-04,2005,1,Mon,CLE,CHW,0,US Cellular Field,56.2,0.000,40615.0,83,0,0,664000,9408576,38141.0
3,2005-04-04,2005,1,Mon,KC,DET,0,Comerica Park,59.0,0.000,41083.0,72,0,0,1000000,4400587,44105.0
4,2005-04-04,2005,1,Mon,MIN,SEA,0,Safeco Field,50.1,0.130,47943.0,63,0,0,927000,4145494,46249.0


In [3]:
# Create dummy model using only h_name variable
homedf = glogs_final.groupby('h_name')['attendance'].mean().sort_values().to_frame()
dummydf = pd.merge(glogs_final, homedf, on='h_name', how='left')
dummydf

# Evaluation metrics for dummy model
mse = mean_squared_error(dummydf['attendance_x'], dummydf['attendance_y'])
rmse = mse**(1/2)
r2 = r2_score(dummydf['attendance_x'], dummydf['attendance_y'])

print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'RMSE: {rmse:.2f}')
print(f'R-squared Score (R²): {r2:.2f}')

Mean Squared Error (MSE): 58894063.76
RMSE: 7674.25
R-squared Score (R²): 0.46


In [3]:
# Data preprocessing

# Drop columns
glogs_final = glogs_final.drop(columns=['date', 'year', 'temp'])

# Binary encode 'day_of_week' and 'precip'
glogs_final['day_of_week'] = glogs_final['day_of_week'].map({'Mon':0, 'Tue':0, 'Wed':0, 'Thu':0, 'Fri':1, 'Sat':1, 'Sun':1}).astype('int')
glogs_final['precip'] = glogs_final['precip'].map(lambda x: 1 if x != 0 else 0)
glogs_final

# Apply OneHotEncoder to 'v_name', 'h_name', 'park_id' columns
encoder = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(sparse_output=False), ['week', 'v_name', 'h_name', 'park_id'])],
    remainder='passthrough'  # Keep other columns if there are any
)

df_encoded = encoder.fit_transform(glogs_final)

# Rename columns
feature_names = encoder.get_feature_names_out()
feature_names = [name.replace("remainder__", "") for name in feature_names]

# Convert the result back to a DataFrame (optional)
df_encoded = pd.DataFrame(df_encoded, columns=feature_names)
df_encoded

,cat__week_1,cat__week_2,cat__week_3,cat__week_4,cat__week_5,cat__week_6,cat__week_7,cat__week_8,cat__week_9,cat__week_10,...,day_of_week,day_night,precip,capacity,prev_year_wins,made_playoffs,won_division,InstagramFollowers,CityPopulation,attendance
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,56937.0,101.0,1.0,1.0,3900000.0,19940274.0,54818.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,45971.0,78.0,0.0,0.0,746000.0,2859024.0,48271.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40615.0,83.0,0.0,0.0,664000.0,9408576.0,38141.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,41083.0,72.0,0.0,0.0,1000000.0,4400587.0,44105.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,47943.0,63.0,0.0,0.0,927000.0,4145494.0,46249.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,50144.0,68.0,0.0,0.0,552000.0,3052498.0,27762.0
29031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,43651.0,63.0,0.0,0.0,1300000.0,6330422.0,36935.0
29032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,41915.0,84.0,0.0,0.0,1600000.0,4648486.0,41445.0
29033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,45494.0,100.0,1.0,1.0,1100000.0,2811927.0,44615.0


In [4]:
# Build random forest model - 9673 cases from 2005-2008
modeldata = df_encoded[:9673]

# Define features (X) and target variable (y)
X = modeldata.drop(columns=['attendance'])
y = modeldata['attendance']

# Split data into training / validation / test (80% train, 10% val, 10% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
#X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


# Initialize the Random Forest regressor
model = RandomForestRegressor(n_estimators=300, random_state=42, max_depth=None, min_samples_leaf=2, min_samples_split=5)
# Train the model
model.fit(X_train, y_train)

# Predict on the val set
y_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE) and R-squared score (R²)
mse = mean_squared_error(y_test, y_pred)
rmse = mse**(1/2)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'RMSE: {rmse:.2f}')
print(f'R-squared Score (R²): {r2:.2f}')

Mean Squared Error (MSE): 23970687.59
RMSE: 4895.99
R-squared Score (R²): 0.81


In [ ]:
# Random Forest hyperparameter tuning
# Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
# Best Parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}


In [25]:
# Build random forest model - 21776 cases from 2005-2013
modeldata = df_encoded[:21776]

# Define features (X) and target variable (y)
X = modeldata.drop(columns=['attendance'])
y = modeldata['attendance']

# Split data into training / test (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Initialize the Random Forest regressor
model = RandomForestRegressor(n_estimators=300, random_state=42, max_depth=None, min_samples_leaf=2, min_samples_split=5)


# Fit the model
model.fit(X_train, y_train)

# Predict on the training set
y_train_pred = model.predict(X_train)

# Calculate training Mean Squared Error (MSE) and R-squared score (R²)
mse = mean_squared_error(y_train, y_train_pred)
rmse = mse**(1/2)
r2 = r2_score(y_train, y_train_pred)

print(f'Training Mean Squared Error (MSE): {mse:.2f}')
print(f'Training RMSE: {rmse:.2f}')
print(f'Training R-squared Score (R²): {r2:.2f}')



# Predict on the test set
y_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE) and R-squared score (R²)
mse = mean_squared_error(y_test, y_pred)
rmse = mse**(1/2)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'RMSE: {rmse:.2f}')
print(f'R-squared Score (R²): {r2:.2f}')

Training Mean Squared Error (MSE): 11162361.04
Training RMSE: 3341.01
Training R-squared Score (R²): 0.90
Mean Squared Error (MSE): 23921382.29
RMSE: 4890.95
R-squared Score (R²): 0.79


In [6]:
# Build Lasso regression model

X = modeldata.drop(columns=["attendance"])
y = modeldata["attendance"]

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit Lasso Regression
lasso = Lasso(alpha=0.1)  # Adjust alpha to control regularization strength
lasso.fit(X_scaled, y)

# Extract feature importance
feature_significance = pd.DataFrame({
    "Variable": X.columns,
    "Lasso_coef": lasso.coef_
})

# Rank features by absolute coefficient value
feature_significance["Abs_Coefficient"] = np.abs(feature_significance["Lasso_coef"])
feature_significance = feature_significance.sort_values(by="Abs_Coefficient", ascending=False)

# Display ranked features
feature_significance[["Variable", "Lasso_coef"]].head(10)

C:\Users\travi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.330e+11, tolerance: 2.481e+08
  model = cd_fast.enet_coordinate_descent(


,Variable,Lasso_coef
72,cat__h_name_NYY,2790.072059
120,day_of_week,2669.071161
67,cat__h_name_LAD,2574.016546
68,cat__h_name_MIA,-2350.715381
80,cat__h_name_TB,-2210.103627
124,prev_year_wins,2064.889531
73,cat__h_name_OAK,-1938.678696
79,cat__h_name_STL,1523.188132
103,cat__park_id_Minute Maid Park,1464.709543
58,cat__h_name_CHC,1427.366809


In [ ]:
# Cross validation for Lasso model
lasso_cv = LassoCV(cv=5).fit(X_scaled, y)
print(f"Optimal alpha: {lasso_cv.alpha_}")

Optimal alpha: 31.640845472570195


In [7]:
# Build Lasso for making predictions
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train Lasso Regression model
lasso = Lasso(alpha=31.640845472570195)
lasso.fit(X_train, y_train)

# Predict on test data
y_pred = lasso.predict(X_test)



lasso_predictions = pd.DataFrame({'Actual':y_test, 'Predictions':y_pred})
lasso_predictions.head()

,Actual,Predictions
9352,21107.0,21002.361363
20640,36590.0,37749.284679
10429,30262.0,27141.231195
21621,30636.0,29003.284402
1503,28971.0,29834.321874


In [8]:
# Evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mse**(1/2)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f'RMSE: {rmse:.2f}')
print(f"R² Score: {r2:.4f}")

Mean Squared Error (MSE): 36430498.7093
RMSE: 6035.77
R² Score: 0.6745


In [78]:
# Convert predictions and actuals to DataFrames with matching indices
train_results = pd.DataFrame({
    'actual': y_train,
    'predicted': y_train_pred
}, index=y_train.index)

test_results = pd.DataFrame({
    'actual': y_test,
    'predicted': y_pred
}, index=y_test.index)


# Combine results
results_df = pd.concat([train_results, test_results]).sort_index().reset_index()
results_df

,index,actual,predicted
0,0,54818.0,53291.284821
1,1,48271.0,38243.313565
2,2,38141.0,27919.193627
3,3,44105.0,19535.328317
4,4,46249.0,37097.580950
...,...,...,...
21771,21771,28315.0,25845.383170
21772,21772,41891.0,32965.223962
21773,21773,41495.0,41686.777405
21774,21774,44808.0,44336.556263


In [100]:
# Error analysis
modeldata = df_encoded[:21776].reset_index()

errordata = pd.merge(pd.merge(modeldata, glogs_final[['h_name', 'park_id']].reset_index(), how='left', on='index'), results_df, how='left', on='index').drop(columns=['index', 'attendance'])

errordata['residual'] = errordata['actual'] - errordata['predicted']
errordata['absresidual'] = np.sqrt(np.square(errordata['residual']))
errordata

,cat__week_1,cat__week_2,cat__week_3,cat__week_4,cat__week_5,cat__week_6,cat__week_7,cat__week_8,cat__week_9,cat__week_10,...,made_playoffs,won_division,InstagramFollowers,CityPopulation,h_name,park_id,actual,predicted,residual,absresidual
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,3900000.0,19940274.0,NYY,Old Yankee Stadium,54818.0,53291.284821,1526.715179,1526.715179
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,746000.0,2859024.0,BAL,Camden Yards,48271.0,38243.313565,10027.686435,10027.686435
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,664000.0,9408576.0,CHW,US Cellular Field,38141.0,27919.193627,10221.806373,10221.806373
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1000000.0,4400587.0,DET,Comerica Park,44105.0,19535.328317,24569.671683,24569.671683
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,927000.0,4145494.0,SEA,Safeco Field,46249.0,37097.580950,9151.419050,9151.419050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,783000.0,6457988.0,MIA,Marlins Park,28315.0,25845.383170,2469.616830,2469.616830
21772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1700000.0,19940274.0,NYM,Citi Field,41891.0,32965.223962,8925.776038,8925.776038
21773,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1600000.0,4648486.0,SF,AT&T Park,41495.0,41686.777405,-191.777405,191.777405
21774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1100000.0,2811927.0,STL,Busch Stadium III,44808.0,44336.556263,471.443737,471.443737


In [111]:
errordata[['h_name', 'residual', 'absresidual']].sort_values(by='absresidual', ascending=False).head(50)

#resids = errordata.groupby('h_name')['absresidual'].mean().sort_values().to_frame()

#resids.plot(kind='bar', y='absresidual')

,h_name,residual,absresidual
9682,TOR,32929.387500,32929.387500
90,OAK,30168.151301,30168.151301
12187,TOR,27366.141893,27366.141893
14522,CLE,27207.620083,27207.620083
4935,TOR,25999.596253,25999.596253
16938,BAL,25481.521243,25481.521243
17027,SEA,24670.759786,24670.759786
3,DET,24569.671683,24569.671683
88,CLE,24246.908447,24246.908447
12135,PIT,-22841.249085,22841.249085
